In [1]:
!nvidia-smi

Wed Oct 30 13:58:08 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:C1:00.0 Off |                    0 |
|  0%   38C    P0    81W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import logging
from PIL import Image
import requests
import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, PaliGemmaConfig, PaliGemmaProcessor, BitsAndBytesConfig
from lavis.common.registry import registry
from lavis.models.base_model import BaseModel
from lavis.common.utils import get_abs_path, is_url, download_cached_file
import numpy as np
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import os
import contextlib
import copy
from tasks.vqa_task_utils import QAOutput

2024-10-30 13:58:13.571161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 13:58:14.744057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2024-10-30 13:58:26,912 - config - PyTorch version 2.2.2 available.
INFO - 2024-10-30 13:58:26,914 - config - TensorFlow version 2.13.1 available.
INFO - 2024-10-30 13:58:26,916 - config - JAX version 0.4.13 available.


In [3]:
from model.paligemma_vqa import PaliGemma_VQA

model_id = "google/paligemma-3b-ft-vqav2-224"
device = "cuda:0"
dtype = torch.bfloat16

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
# print("image", np.array(image))

samples = {
    "image_raw": [image, image],
    "text_input_raw": ["caption es", "caption es: "],
}
with torch.inference_mode():
    model = PaliGemma_VQA(model_id=model_id, dtype=dtype).to(device)
    output = model.predict_answers(samples)
    print(output)

model_id google/paligemma-3b-ft-vqav2-224


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PaliGemmaConfig {
  "_name_or_path": "google/paligemma-3b-ft-vqav2-224",
  "architectures": [
    "PaliGemmaForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "eos_token_id": 1,
  "hidden_size": 2048,
  "ignore_index": -100,
  "image_token_index": 257152,
  "model_type": "paligemma",
  "pad_token_id": 0,
  "projection_dim": 2048,
  "text_config": {
    "hidden_size": 2048,
    "intermediate_size": 16384,
    "model_type": "gemma",
    "num_attention_heads": 8,
    "num_hidden_layers": 18,
    "num_image_tokens": 256,
    "num_key_value_heads": 1,
    "torch_dtype": "float32",
    "vocab_size": 257216
  },
  "torch_dtype": "bfloat16",
  "transformers_version": "4.43.0.dev0",
  "vision_config": {
    "hidden_size": 1152,
    "intermediate_size": 4304,
    "model_type": "siglip_vision_model",
    "num_attention_heads": 16,
    "num_hidden_layers": 27,
    "num_image_tokens": 256,
    "patch_size": 14,
    "projection_dim": 2048,
    "projector_hidden_act": "gelu_fast",
    "vision_use_

In [144]:
def attn_scores(samples):
    with torch.inference_mode():
        outputs = model.return_all_outputs(samples)
    tokens = model.processor(text=samples["text_input_raw"], images=samples["image_raw"], suffix=samples["multiple_choice_answer"], return_tensors="pt", padding="longest").to(dtype).to(device)
    
    print("Model Predictions")
    print(model.predict_answers(samples))
    
    for i in range(len(samples['image_raw'])):
        print("===== Sample", i)
        print(f"Image: {samples['image_raw'][i]}")
        print(f"Question: {samples['text_input_raw'][i]}")
        attn = outputs.attentions[-1][i].mean(dim=0)

        img_token_idx = tokens['input_ids'][i] == 257152
        suffix_token_idx = tokens['token_type_ids'][i] == 1
        prefix_token_idx = (img_token_idx == False) & (suffix_token_idx == False)

        img_attn_score = attn[img_token_idx][:, img_token_idx].mean().item()
        img_pre_attn_score = attn[img_token_idx][:, prefix_token_idx].mean().item()
        pre_img_attn_score = attn[prefix_token_idx][:, img_token_idx].mean().item()
        pre_attn_score = attn[prefix_token_idx][:, prefix_token_idx].mean().item()
        
        print("image token, txt/img", (attn[img_token_idx][:, prefix_token_idx].sum(dim=1) / attn[img_token_idx][:, img_token_idx].sum(dim=1)).mean().item())
        print("text token, txt/img", (attn[prefix_token_idx][:, prefix_token_idx].sum(dim=1) / attn[prefix_token_idx][:, img_token_idx].sum(dim=1)).mean().item())


#         print("img_attn_score:", img_attn_score)
#         print("img_pre_attn_score:", img_pre_attn_score)

#         print("pre_attn_score:", pre_attn_score)
#         print("pre_img_attn_score:", pre_img_attn_score)
        
#         # top right
#         print("top right:", img_pre_attn_score / img_attn_score)
        
#         # bottom left
#         print("bottom left:", pre_attn_score / pre_img_attn_score)
    
    
url_2 = "https://drive.google.com/uc?export=download&id=1K39drwi_HrPtRkMf-RTB3zgzGNIBBEyr"
image_2 = Image.open(requests.get(url_2, stream=True).raw).convert("RGB")

samples = {
    "image_raw": [image, image, image, image_2, image_2, image_2, image, image, image, image, image_2],
    "text_input_raw": ["Is there a blue volkswagen car?", "Is there a car?", "Is there a giraffe?", "Is there a car?", "Is there a giraffe?", "Is there any tree without leaves in this picture?", "What is the brand of the car?", "What is the color of the car?", "What is the color of the sky?", "Who is the president of the USA?", "Who is the president of the USA?"],
    "multiple_choice_answer": ['yes', 'yes', 'no', 'no', 'yes', 'no', 'volkswagen beetle', 'volkswagen beetle', 'volkswagen beetle', 'volkswagen beetle', 'volkswagen beetle']
}        

attn_scores(samples)

Model Predictions
['yes', 'yes', 'no', 'no', 'yes', 'no', 'volkswagen', 'blue', 'blue', 'obama', 'obama']
===== Sample 0
Image: <PIL.Image.Image image mode=RGB size=640x480 at 0x7F8996277FD0>
Question: Is there a blue volkswagen car?
image token, txt/img 0.85546875
text token, txt/img 2.453125
===== Sample 1
Image: <PIL.Image.Image image mode=RGB size=640x480 at 0x7F8996277FD0>
Question: Is there a car?
image token, txt/img 0.76953125
text token, txt/img 1.515625
===== Sample 2
Image: <PIL.Image.Image image mode=RGB size=640x480 at 0x7F8996277FD0>
Question: Is there a giraffe?
image token, txt/img 0.78125
text token, txt/img 2.046875
===== Sample 3
Image: <PIL.Image.Image image mode=RGB size=640x480 at 0x7F89919EEF40>
Question: Is there a car?
image token, txt/img 0.8984375
text token, txt/img 2.3125
===== Sample 4
Image: <PIL.Image.Image image mode=RGB size=640x480 at 0x7F89919EEF40>
Question: Is there a giraffe?
image token, txt/img 0.88671875
text token, txt/img 1.796875
===== Sampl

In [128]:
def attn_scores_mat(samples):

    # Shape of attention matrix [batch_size, seq_len, seq_len]
    attn = outputs.attentions[-1].mean(dim=1)  # shape: [batch_size, seq_len, seq_len]

    # Define the token indices for rows and columns
    img_token_idx = (tokens['input_ids'] == 257152)              # shape: [batch_size, seq_len]
    prefix_token_idx = (tokens['input_ids'] != 257152) & (tokens['token_type_ids'] == 0)  # shape: [batch_size, seq_len] 
    
    img_attn_idx = img_token_idx.unsqueeze(-1) * img_token_idx.unsqueeze(1)
    img_pre_attn_idx = img_token_idx.unsqueeze(-1) * prefix_token_idx.unsqueeze(1)
    pre_img_attn_idx = prefix_token_idx.unsqueeze(-1) * img_token_idx.unsqueeze(1)
    pre_attn_idx = prefix_token_idx.unsqueeze(-1) * prefix_token_idx.unsqueeze(1)

    
#     print(img_ratio)
#     print(txt_ratio)

attn_scores_mat(samples)

torch.Size([11, 270, 270])
tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0],
        ...,
        [  0, 256, 256,  ...,   0,   0,   0],
        [  0, 256, 256,  ...,   0,   0,   0],
        [  0, 256, 256,  ...,   0,   0,   0]], device='cuda:0')


In [ ]:
attn = outputs.attentions[-1].mean(dim=1)  # shape: [batch_size, seq_len, seq_len]
img_token_idx = tokens['input_ids'] == 257152                # shape: [batch_size, seq_len]
suffix_token_idx = tokens['token_type_ids'] == 1             # shape: [batch_size, seq_len]
prefix_token_idx = (~img_token_idx) & (~suffix_token_idx)    # shape: [batch_size, seq_len]

# Calculate img_ratio for each sample in batch
img_attn_sum = attn[img_token_idx][:, prefix_token_idx].sum(dim=1)
img_self_sum = attn[img_token_idx][:, img_token_idx].sum(dim=1)
img_ratio = (img_attn_sum / img_self_sum).mean(dim=1)

# Calculate txt_ratio for each sample in batch
txt_attn_sum = attn[prefix_token_idx][:, prefix_token_idx].sum(dim=1)
txt_self_sum = attn[prefix_token_idx][:, img_token_idx].sum(dim=1)
txt_ratio = (txt_attn_sum / txt_self_sum).mean(dim=1)

In [41]:
import torch.nn.functional as F

def attn_align(samples):
    KLDivLoss = torch.nn.KLDivLoss(reduction='batchmean')
    with torch.inference_mode():
        outputs = model.return_all_outputs(samples)
    tokens = model.processor(text=samples["text_input_raw"], images=samples["image_raw"], suffix=samples["multiple_choice_answer"], return_tensors="pt", padding="longest").to(dtype).to(device)
    
    print("Model Predictions")
    print(model.predict_answers(samples))
    
    for i in range(len(samples['image_raw'])):
        print("===== Sample", i)
        print(f"Image: {samples['image_raw'][i]}")
        print(f"Question: {samples['text_input_raw'][i]}")
        attn = outputs.attentions[-1][i].mean(dim=0)

        img_token_idx = tokens['input_ids'][i] == 257152
        suffix_token_idx = tokens['token_type_ids'][i] == 1
        prefix_token_idx = (img_token_idx == False) & (suffix_token_idx == False)

        img_attn = attn[img_token_idx][:, img_token_idx]
#         assert (attn[img_token_idx][:, img_token_idx] != attn[:, img_token_idx][img_token_idx]).sum().sum() == 0
        img_pre_attn = attn[img_token_idx][:, prefix_token_idx]
        pre_img_attn = attn[prefix_token_idx][:, img_token_idx]
        pre_attn = attn[prefix_token_idx][:, prefix_token_idx]
        
        img2txt = pre_img_attn @ img_attn @ pre_img_attn.T  # pre_img_attn.T
        txt2img = img_pre_attn @ pre_attn @ img_pre_attn.T  # img_pre_attn.T
        
        print("img_attn", img_attn.sum(dim=0))  # should calculate for each token mean
        print("pre_attn", pre_attn.sum(dim=0))
        print("img_pre_attn", img_pre_attn)
        print("pre_img_attn", pre_img_attn)


        print("img_attn_score:", img_attn.sum().item())
        print("txt2img_score:", txt2img.sum().item())

        print("pre_attn_score:", pre_attn.sum().item())
        print("img2txt_score:", img2txt.sum().item())
        
        # top right
        print("top right:", img2txt.mean().item() / pre_attn.mean().item())
        
        # bottom left
        print("bottom left:", txt2img.mean().item() / img_attn.mean().item())
        
        txt_attn = pre_attn
        pseudo_txt_attn = img2txt
        img_attn = img_attn
        pseudo_img_attn = txt2img
        
#         print(txt_attn)
#         print(pseudo_txt_attn)
        
#         print(F.softmax(txt_attn, dim=1))
#         print(F.softmax(pseudo_txt_attn, dim=1))
        
        txt_kl = KLDivLoss(F.log_softmax(txt_attn + 1e-6, dim=1), F.softmax(pseudo_txt_attn, dim=1)) + KLDivLoss(F.log_softmax(pseudo_txt_attn + 1e-6, dim=1), F.softmax(txt_attn, dim=1))
        img_kl = KLDivLoss(F.log_softmax(img_attn + 1e-6, dim=1), F.softmax(pseudo_img_attn, dim=1)) + KLDivLoss(F.log_softmax(pseudo_img_attn + 1e-6, dim=1), F.softmax(img_attn, dim=1))
        
#         print(txt_attn.shape)
        
        print("txt_kl:", txt_kl.item())
        print("img_kl:", img_kl.item())

attn_align(samples)

Model Predictions
['yes', 'yes', 'no', 'no', 'yes', 'no', 'volkswagen', 'blue', 'blue', 'obama', 'obama']
===== Sample 0
Image: <PIL.Image.Image image mode=RGB size=640x480 at 0x7F8996277FD0>
Question: Is there a blue volkswagen car?
img_attn tensor([0.2773, 0.4082, 0.2031, 0.2891, 0.1582, 0.1211, 0.4004, 0.2617, 0.1494,
        0.1836, 0.4023, 0.2871, 0.2295, 0.3887, 0.3066, 0.3867, 0.3711, 0.1787,
        0.2715, 0.3555, 0.1533, 0.2158, 0.1045, 0.1172, 0.2578, 0.2852, 0.2617,
        0.3379, 0.1514, 0.2871, 0.2021, 0.3242, 0.4980, 0.2930, 0.5156, 0.2295,
        0.2715, 0.1914, 0.8438, 0.3086, 0.3965, 0.1279, 0.3965, 0.1338, 0.5078,
        1.0156, 0.2754, 2.0469, 4.9688, 0.4004, 0.5742, 0.2949, 0.0996, 0.2480,
        0.4180, 0.1660, 0.4277, 0.1206, 0.2490, 0.1738, 0.4551, 0.1006, 0.4375,
        0.1504, 0.2773, 1.1719, 0.4648, 0.4102, 0.2676, 0.1689, 0.2734, 0.1562,
        0.5039, 0.3574, 0.1807, 0.2383, 0.3691, 0.1846, 0.1807, 0.4609, 0.4355,
        0.1748, 0.7188, 0.3340, 0.347

In [57]:
with torch.inference_mode():
    outputs = model.return_all_outputs(samples)

i=0
attn = outputs.attentions[-1][i].mean(dim=0)
tokens = model.processor(text=samples["text_input_raw"], images=samples["image_raw"], suffix=samples["multiple_choice_answer"], return_tensors="pt", padding="longest").to(dtype).to(device)

img_token_idx = tokens['input_ids'][i] == 257152
suffix_token_idx = tokens['token_type_ids'][i] == 1
prefix_token_idx = (img_token_idx == False) & (suffix_token_idx == False)

img_attn = attn[img_token_idx][:, img_token_idx]
img_pre_attn = attn[img_token_idx][:, prefix_token_idx]
pre_img_attn = attn[prefix_token_idx][:, img_token_idx]
pre_attn = attn[prefix_token_idx][:, prefix_token_idx]

print(img_attn)
print((img_attn != img_attn.T).sum().sum())

tensor([[0.0269, 0.0549, 0.0048,  ..., 0.0029, 0.0011, 0.0063],
        [0.0127, 0.0815, 0.0228,  ..., 0.0033, 0.0011, 0.0070],
        [0.0115, 0.0586, 0.0830,  ..., 0.0041, 0.0007, 0.0056],
        ...,
        [0.0004, 0.0006, 0.0003,  ..., 0.0282, 0.0176, 0.0132],
        [0.0011, 0.0028, 0.0007,  ..., 0.0143, 0.0309, 0.0184],
        [0.0003, 0.0009, 0.0002,  ..., 0.0099, 0.0063, 0.0874]],
       device='cuda:0', dtype=torch.bfloat16)
tensor(65190, device='cuda:0')


In [64]:
with torch.inference_mode():
    outputs = model.return_all_outputs(samples)

i=0
attn = outputs.attentions[-1][i].mean(dim=0)
attn.sum(dim=1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

tensor([[0.0000, 0.1416, 0.1035,  ..., 0.0234, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1348,  ..., 0.0275, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0243, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0100, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.1416, 0.2158, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0452, 0.3496, 0.1992]],
       device='cuda:0', dtype=torch.bfloat16)

In [4]:
samples = {
    "image_raw": [image, image],
    "text_input_raw": ["What is the color of the sky?", "What is the color of the sky?"],
    "multiple_choice_answer": ['volkswagen beetle', 'volkswagen beetle']
}
with torch.inference_mode():
    outputs = model.return_all_outputs(samples)
outputs

PaliGemmaCausalLMOutputWithPast(loss=tensor(7.0863, device='cuda:0'), logits=tensor([[[ -7.8125,  16.5000,   1.7031,  ...,  -5.4062,  -5.4062,  -5.4688],
         [ -8.0625,  14.3750,   1.5703,  ...,  -5.4375,  -5.4375,  -5.4688],
         [ -1.6875,  11.8750, -10.2500,  ...,  -1.3203,  -1.3047,  -1.3203],
         ...,
         [ -5.9375,  17.3750,  -8.9375,  ...,  -5.0938,  -5.0938,  -5.0938],
         [ -8.2500,  15.6875,  -6.7812,  ...,  -6.7812,  -6.7812,  -6.7812],
         [ -3.8281,  16.8750,   1.9688,  ...,  -3.3438,  -3.3750,  -3.3906]],

        [[ -7.8125,  16.5000,   1.7031,  ...,  -5.4062,  -5.4062,  -5.4688],
         [ -8.0625,  14.3750,   1.5703,  ...,  -5.4375,  -5.4375,  -5.4688],
         [ -1.6875,  11.8750, -10.2500,  ...,  -1.3203,  -1.3047,  -1.3203],
         ...,
         [ -5.9375,  17.3750,  -8.9375,  ...,  -5.0938,  -5.0938,  -5.0938],
         [ -8.2500,  15.6875,  -6.7812,  ...,  -6.7812,  -6.7812,  -6.7812],
         [ -3.8281,  16.8750,   1.9688,  ..., 

In [5]:
model.predict_answers(samples)

['blue', 'blue']

In [6]:
outputs.attentions[0].shape

torch.Size([2, 8, 269, 269])

In [7]:
len(outputs.attentions)

18

In [17]:
print(model)

PaliGemma_VQA(
  (model): PaliGemmaForConditionalGeneration(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(1024, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (mlp): SiglipMLP(
                (activation_fn)

In [8]:
for i in range(len(outputs.attentions)):
    print(outputs.attentions[i].shape)

torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])
torch.Size([2, 8, 269, 269])


In [9]:
# last layer attention, first example
outputs.attentions[-1][0].mean(dim=0).shape

torch.Size([269, 269])

In [141]:
print(outputs.attentions[-1].shape)

torch.Size([11, 8, 270, 270])


In [11]:
tokens = model.processor(text=samples["text_input_raw"], images=samples["image_raw"], suffix=samples["multiple_choice_answer"], return_tensors="pt", padding="longest").to(dtype).to(device)

In [12]:
tokens

{'input_ids': tensor([[257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         25715

In [21]:
# image token idx
img_token_idx = tokens['input_ids'][0] == 257152
sum(img_token_idx), img_token_idx

(tensor(256, device='cuda:0'),
 tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True

In [22]:
# suffix tokens index
suffix_token_idx = tokens['token_type_ids'][0] == 1
sum(suffix_token_idx), suffix_token_idx

(tensor(3, device='cuda:0'),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, 

In [26]:
# prefix tokens index
prefix_token_idx = (img_token_idx == False) & (suffix_token_idx == False)
sum(prefix_token_idx), prefix_token_idx

(tensor(10, device='cuda:0'),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False,

In [50]:
print(model.predict_answers(samples)[0])
attn = outputs.attentions[-1][0].mean(dim=0)
print(attn[img_token_idx][:, img_token_idx].sum().sum())
print(attn[img_token_idx][:, prefix_token_idx].sum().sum())
# top right
print(attn[img_token_idx][:, prefix_token_idx].sum().sum() / attn[img_token_idx][:, img_token_idx].sum().sum())
print(attn[prefix_token_idx][:, prefix_token_idx].sum().sum())
print(attn[prefix_token_idx][:, img_token_idx].sum().sum())
# bottom left
print(attn[prefix_token_idx][:, img_token_idx].sum().sum() / attn[prefix_token_idx][:, prefix_token_idx].sum().sum())

blue
tensor(148., device='cuda:0', dtype=torch.bfloat16)
tensor(107.5000, device='cuda:0', dtype=torch.bfloat16)
tensor(0.7266, device='cuda:0', dtype=torch.bfloat16)
tensor(7.4688, device='cuda:0', dtype=torch.bfloat16)
tensor(2.5312, device='cuda:0', dtype=torch.bfloat16)
tensor(0.3398, device='cuda:0', dtype=torch.bfloat16)


In [13]:
tokens.pixel_values.shape

torch.Size([2, 3, 224, 224])

In [14]:
text_tokens = tokens["input_ids"]
text_sequence_length = text_tokens.size(1)
text_sequence_length

269

In [15]:
sum(tokens["token_type_ids"][0] != 0)

tensor(3, device='cuda:0')

In [16]:
tokens["labels"][0]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 

In [17]:
sum(np.array(tokens["labels"][0].cpu()) != -100)

3

In [55]:
#   "bos_token_id": 2,
#   "eos_token_id": 1,
# "ignore_index": -100,
#   "image_token_index": 257152,
#   "pad_token_id": 0,
# 448: "num_image_tokens": 1024, 224: "num_image_tokens": 256,
text_tokens, text_tokens.shape

(tensor([[     0,      0, 257152,  ...,  51108,  61936,      1],
         [257152, 257152, 257152,  ...,  51108,  61936,      1]],
        device='cuda:0'),
 torch.Size([2, 1033]))

In [64]:
text_tokens

tensor([[     0,      0, 257152,  ...,  51108,  61936,      1],
        [257152, 257152, 257152,  ...,  51108,  61936,      1]],
       device='cuda:0')

In [59]:
sum(text_tokens[0] != 257152), sum(text_tokens[1] != 257152)

(tensor(9, device='cuda:0'), tensor(9, device='cuda:0'))

In [62]:
tokens["input_ids"][:, 1024:]

tensor([[257152, 257152,      2,  21209,    875,    108,  51108,  61936,      1],
        [     2,  21209,    875, 235292, 235248,    108,  51108,  61936,      1]],
       device='cuda:0')

In [57]:
tokens["input_ids"][:, 256:].shape

torch.Size([2, 777])